In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib
import seaborn as sns
import csv
import re 
from matplotlib import pyplot as plt
from sklearn import ensemble, cross_validation, learning_curve, metrics , tree, naive_bayes, utils
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.datasets import load_digits, load_breast_cancer, load_iris
from sklearn.cross_validation import cross_val_score, KFold, LeaveOneOut, ShuffleSplit, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, BaggingRegressor
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans, AgglomerativeClustering, FeatureAgglomeration
from sklearn.preprocessing import StandardScaler, scale, OneHotEncoder
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction import DictVectorizer
matplotlib.style.use('ggplot')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_train = pd.read_csv('train2.csv')
df_test = pd.read_csv('test2.csv')

## Feature engenering

In [3]:
df_train.head()

,year,day,team1,team2,score1,score2,target
0,2998,19,317,131,336,278,True
1,2998,28,61,29,301,259,True
2,2998,28,110,141,359,267,True
3,2998,28,352,146,309,410,False
4,2998,28,229,91,332,220,True


In [32]:
means_score = {}
for index_team in list(df_train.team1.unique()):
    df_for_team1 = df_train[df_train['team1'] == index_team]
    df_for_team2 = df_train[df_train['team2'] == index_team]
    means_score[index_team] = [int(df_for_team1.score1.mean()), int(df_for_team2.score2.mean())]
#print(means_score)
#for c in means_score:
#    print(means_score[c])

In [33]:
teams_not_train = []
for index_team in list(df_test.team1.unique()):
    if index_team not in list(df_train.team1.unique()):
        teams_not_train.append(index_team)
for index_team in list(df_test.team2.unique()):
    if index_team not in list(df_train.team1.unique()):
        teams_not_train.append(index_team)

In [34]:
teams_not_train

[113, 162, 130, 162, 1, 113, 130]

In [35]:
for index_team in teams_not_train:
    means_score[index_team] = [df_train.score1.mean(), df_train.score2.mean()]

In [36]:
features_to_concat_train = []
for year, day, team1, team2, _, __, ___ in df_train.values:
    features_to_concat_train.append([means_score[team1][0], means_score[team2][1]])
features_to_concat_test = []
for Id, year, team1, team2 in df_test.values:
    features_to_concat_test.append([means_score[team1][0], means_score[team2][1]])

In [37]:
df_feature_to_concat_train = pd.DataFrame(data=features_to_concat_train, columns=['mean_home_score', 'mean_away_score'])

In [39]:
df_feature_to_concat_test = pd.DataFrame(data=features_to_concat_test, columns=['mean_home_score', 'mean_away_score'])

## Средний счет по версусу оставим на потом

In [40]:
means_score_team1_vs_team2 = {}
for index_team in list(df_train.team1.unique()):
    df_1 = df_train[df_train['team1'] = index_team]
print(means_score)

SyntaxError: invalid syntax (<ipython-input-40-e2e27904b906>, line 3)

## Считаем процент побед дома и в гостях

In [48]:
percentage_wins = {}
for index_team in list(df_train.team1.unique()):
    df_for_team_home = df_train[df_train['team1'] == index_team]
    df_for_team_away = df_train[df_train['team2'] == index_team]
    percentage_wins[index_team] = [df_for_team_home.target.mean(), 1 - df_for_team_away.target.mean()]
print(percentage_wins)

{2: [0.38144329896907214, 0.39446366782006925], 3: [0.50501672240802675, 0.57099697885196377], 4: [0.64080459770114939, 0.58878504672897192], 5: [0.40404040404040403, 0.41818181818181821], 6: [0.47931034482758622, 0.44117647058823528], 7: [0.41784037558685444, 0.47549019607843135], 8: [0.35333333333333333, 0.32894736842105265], 10: [0.46153846153846156, 0.46302250803858525], 11: [0.49832775919732442, 0.47741935483870968], 12: [0.74096385542168675, 0.73746312684365778], 13: [0.52890173410404628, 0.50662251655629142], 14: [0.49666666666666665, 0.53205128205128205], 15: [0.28985507246376813, 0.25570776255707761], 16: [0.64814814814814814, 0.58139534883720922], 17: [0.48601398601398599, 0.47112462006079026], 19: [0.25454545454545452, 0.31309904153354629], 20: [0.51419558359621453, 0.51384615384615384], 22: [0.48909657320872274, 0.52580645161290329], 23: [0.53605015673981193, 0.52597402597402598], 24: [0.44827586206896552, 0.54666666666666663], 25: [0.65463917525773196, 0.59817351598173518]

In [49]:
for index_team in teams_not_train:
    percentage_wins[index_team] = [df_train.target.mean(), 1 - df_train.target.mean()]

In [50]:
percentage_of_wins_train = []
for year, day, team1, team2, _, __, ___ in df_train.values:
    percentage_of_wins_train.append([percentage_wins[team1][0], percentage_wins[team2][1]])
percentage_of_wins_test = []
for Id, year, team1, team2 in df_test.values:
    percentage_of_wins_test.append([percentage_wins[team1][0], percentage_wins[team2][1]])

In [51]:
df_percentage_of_wins_train = pd.DataFrame(data=percentage_of_wins_train, columns=['percentage_wins_home', 'percentage_wins_away'])

In [52]:
df_percentage_of_wins_test = pd.DataFrame(data=percentage_of_wins_test, columns=['percentage_wins_home', 'percentage_wins_away'])

In [55]:
df_percentage_of_wins_train

,percentage_wins_home,percentage_wins_away
0,0.688073,0.640118
1,0.503333,0.556962
2,0.526786,0.455090
3,0.582237,0.803030
4,0.690184,0.479624
5,0.523529,0.610942
6,0.434921,0.642599
7,0.632432,0.525974
8,0.638418,0.535714
9,0.503333,0.543807


In [56]:
clf = xgb.XGBClassifier()

In [57]:
clf.get_params()

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'silent': True,
 'subsample': 1}

In [59]:
df_317 = df_train[df_train['team1'] == 317]
df_317_131 = df_317[df_317['team2'] == 131]
df_317_131

,year,day,team1,team2,score1,score2,target
0,2998,19,317,131,336,278,True


In [60]:
df_317 = df_train[df_train['team1'] == 131]
df_317_131 = df_317[df_317['team2'] == 317]
df_317_131

,year,day,team1,team2,score1,score2,target
